# Calling Necessary Libraries AND Getting Today's Date

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
from scipy.stats import poisson
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Get today's date
given_date = "2024-12-11" #year-month-day
threshold = 70 #threshold for percentages to highlight in final dataframe (between 1 and 100)

wanted_leagues = ['argentina', 'austria', 'belgium', 'brazil', 'denmark', 
                  'england', 'england2', 'england3', 'england4', 'england5', 'france', 'france2',
                  'germany', 'germany2', 'greece', 'italy', 'italy2',
                   'mexico', 'netherlands', 'norway','poland', 'portugal', 'portugal2',
                  'scotland', 'scotland2', 'scotland3', 'scotland4',
                  'spain', 'spain2', 'sweden', 'switzerland', 'turkey', 'usa']

In [2]:
#Calculating Days between given date and today
# Today's date
today = datetime.now().date()

# Specific date
specific_date = datetime.strptime(given_date, "%Y-%m-%d").date()

# Calculate the difference in days
difference = specific_date - today

# Add one day to the difference
days_until_specific_date = difference.days + 1

# Scraping Today's Matches and Leagues

In [3]:
URL = "https://www.soccerstats.com/matches.asp?matchday=" + str(days_until_specific_date) + "&listing=2"
page = requests.get(URL)
liqa = []
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="btable")
sth = results.find_all("tr", attrs={'height': '34'})

In [4]:
# Initialize lists to store extracted data
league_list = []
home_team_list = []
away_team_list = []

# Iterate through each <tr> element
for tr_element in sth:
    # Find the <td> element with sorttable_customkey attribute
    td_element = tr_element.find('td', attrs={'sorttable_customkey': True})
    
    # Check if <td> element exists and contains an <img> element with the alt attribute
    if td_element and td_element.find('img', alt=True):
        # Extract league info
        league = td_element['sorttable_customkey']
        if league in wanted_leagues:
            league_list.append(league)
        
        # Find <td> elements with class "steam"
        td_elements_steam = tr_element.find_all('td', class_='steam')
        
        # Extract home and away team info
        if len(td_elements_steam) == 2 and league in wanted_leagues:
            home_team = td_elements_steam[0].get_text(strip=True)
            away_team = td_elements_steam[1].get_text(strip=True)
            home_team_list.append(home_team)
            away_team_list.append(away_team)
        else:
            continue
    else:
        continue

# Create DataFrame
matches = pd.DataFrame({
    'league': league_list,
    'home_team': home_team_list,
    'away_team': away_team_list
})

# Display DataFrame
matches.head()

,league,home_team,away_team


# Converting Date and Collecting Leagues for Analysis

In [5]:
from dateutil import parser

day_abbreviations = {0: "Mo", 1: "Tu", 2: "We", 3: "Th", 4: "Fr", 5: "Sa", 6: "Su"}
given_date_parsed = parser.parse(given_date)

# Manually format the day to remove leading zeros
day = given_date_parsed.day
month = given_date_parsed.strftime('%b')

# Format the date as "Su 1 Oct" or "Tu 10 Oct" without leading zero for single-digit days
formatted_date = f"{day_abbreviations[given_date_parsed.weekday()]} {day} {month}"

# Scraping the Web for the League Statistics

In [6]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = wanted_leagues#matches['league'].unique().tolist()
next_matches = pd.DataFrame()

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    next_matches = pd.concat([next_matches, next_df], ignore_index = True)
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']
combined_df = pd.concat([final.head(), final.tail()])
combined_df

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG
0,Fr 10 May,Argentina,Sarmiento,Instituto,1 - 2,(0-1),1,2,3,0,1,1
1,Sa 11 May,Argentina,Argentinos Jrs,Rosario Central,3 - 2,(2-2),3,2,5,2,2,4
2,Sa 11 May,Argentina,Newells,Platense,2 - 0,(0-0),2,0,2,0,0,0
3,Sa 11 May,Argentina,Huracan,Defensa y J.,3 - 1,(1-0),3,1,4,1,0,1
4,Sa 11 May,Argentina,Godoy Cruz,Barracas C.,0 - 1,(0-0),0,1,1,0,0,0
5610,Su 20 Oct,Usa,Houston Dynamo,LA Galaxy,2 - 1,(1-0),2,1,3,1,0,1
5611,Su 20 Oct,Usa,Los Angeles FC,SJ Earthquakes,3 - 1,(0-1),3,1,4,0,1,1
5612,Su 20 Oct,Usa,Minnesota Utd,St. Louis City,4 - 1,(1-0),4,1,5,1,0,1
5613,Su 20 Oct,Usa,Real Salt Lake,Vancouver,2 - 1,(0-0),2,1,3,0,0,0
5614,Su 20 Oct,Usa,Seattle,Portland,1 - 1,(1-0),1,1,2,1,0,1


In [7]:
next_leagues = next_matches['League'].unique().tolist()
pd.concat([next_matches.head(), next_matches.tail()])

,Date,League,Home,Away,FT,HT
0,We 11 Dec,England2,Cardiff City,Preston,19:45,
1,We 11 Dec,England2,Hull City,Watford,19:45,
2,We 11 Dec,England2,Millwall,Sheffield Utd,19:45,
3,We 11 Dec,England2,QP Rangers,Oxford Utd,19:45,
4,We 11 Dec,England2,West Brom,Coventry City,20:00,
0,We 11 Dec,England2,Cardiff City,Preston,19:45,
1,We 11 Dec,England2,Hull City,Watford,19:45,
2,We 11 Dec,England2,Millwall,Sheffield Utd,19:45,
3,We 11 Dec,England2,QP Rangers,Oxford Utd,19:45,
4,We 11 Dec,England2,West Brom,Coventry City,20:00,


# Calculating Functions Needed for Dixon-Coles Model

In [8]:
from scipy.optimize import minimize
from scipy.stats import poisson

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))



def solve_parameters(dataset, half_or_full = 'full', debug = False, init_vals=None, options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['Home'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['Away'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Something's not right")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0, 1.0]) # rho (score correction), gamma (home advantage)
                                     ))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        if half_or_full == 'full':
            log_like = [dc_log_like(row.FTHG, row.FTAG, score_coefs[row.Home], defend_coefs[row.Home],
                        score_coefs[row.Away], defend_coefs[row.Away], rho, gamma) for row in dataset.itertuples()]
        elif half_or_full == 'half':
            log_like = [dc_log_like(row.HTHG, row.HTAG, score_coefs[row.Home], defend_coefs[row.Home],
                        score_coefs[row.Away], defend_coefs[row.Away], rho, gamma) for row in dataset.itertuples()]

        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints, **kwargs)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))


# Calculating Lambda Values for Dixon-Coles Model

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
stats_df = pd.DataFrame()
full_time_models = []
half_time_models = []

for league in next_leagues:
    league_df = final[final['League'] == league.capitalize()]
    
    full_time_estimates = solve_parameters(league_df, half_or_full = 'full')
    full_time_models.append(full_time_estimates)

    half_time_estimates = solve_parameters(league_df, half_or_full = 'half')
    half_time_models.append(half_time_estimates)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 599.016186492993
            Iterations: 81
            Function evaluations: 4230
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 401.1070076444822
            Iterations: 90
            Function evaluations: 4762
            Gradient evaluations: 90


In [10]:
full_time_models[-1]

{'attack_Blackburn': 1.0569545949812793,
 'attack_Bristol City': 1.062548617801119,
 'attack_Burnley': 1.0038086997786368,
 'attack_Cardiff City': 0.749627228667564,
 'attack_Coventry City': 1.1096738101041685,
 'attack_Derby County': 0.9399877017381421,
 'attack_Hull City': 0.7076080557670731,
 'attack_Leeds Utd': 1.335071695696118,
 'attack_Luton Town': 0.9394457435237813,
 'attack_Middlesbrough': 1.3465498357945322,
 'attack_Millwall': 0.8621169134281195,
 'attack_Norwich City': 1.4120366100567772,
 'attack_Oxford Utd': 0.9690914487383379,
 'attack_Plymouth': 0.8117950698459804,
 'attack_Portsmouth': 1.0790358595300173,
 'attack_Preston': 0.7947954158106343,
 'attack_QP Rangers': 0.7664130987935414,
 'attack_Sheffield Utd': 1.1448691015375152,
 'attack_Sheffield Wed': 1.0235244566478052,
 'attack_Stoke City': 0.88504511302361,
 'attack_Sunderland': 1.2151138469773197,
 'attack_Swansea City': 0.813069589878981,
 'attack_Watford': 1.1803175267078085,
 'attack_West Brom': 0.95061089603

# Calculating Probability Matrices for Half/Full Time

In [11]:
#First Function needs work to make it more understandable and a df rather than matrix!
def dixon_coles_simulate_match(params_dict, homeTeam, awayTeam, max_goals=10):
    team_avgs = [np.exp(params_dict['attack_'+homeTeam] + params_dict['defence_'+awayTeam] + params_dict['home_adv']),
                 np.exp(params_dict['defence_'+homeTeam] + params_dict['attack_'+awayTeam])]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in team_avgs]
    output_matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    correction_matrix = np.array([[rho_correction(home_goals, away_goals, team_avgs[0],
                                                   team_avgs[1], params_dict['rho']) for away_goals in range(2)]
                                   for home_goals in range(2)])
    output_matrix[:2,:2] = output_matrix[:2,:2] * correction_matrix
    return output_matrix

full_time_matrices = []
half_time_matrices = []

for i in range(len(next_matches)):
    my_league = next_matches['League'].iloc[i]
    league_index = next_leagues.index(my_league)
    ft_match_score_matrix = dixon_coles_simulate_match(full_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 8)
    ht_match_score_matrix = dixon_coles_simulate_match(half_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 4)
    full_time_matrices.append(ft_match_score_matrix)
    half_time_matrices.append(ht_match_score_matrix)

full_time_matrices[0]

array([[1.25174986e-01, 9.19162909e-02, 5.47006607e-02, 1.83292954e-02,
        4.60638498e-03, 9.26114493e-04, 1.55162898e-04, 2.22825196e-05,
        2.79994024e-06],
       [1.14944015e-01, 1.49463541e-01, 6.66229812e-02, 2.23242697e-02,
        5.61037283e-03, 1.12796642e-03, 1.88981536e-04, 2.71391216e-05,
        3.41020316e-06],
       [8.02983547e-02, 8.07199900e-02, 4.05719197e-02, 1.35949857e-02,
        3.41659277e-03, 6.86906563e-04, 1.15085569e-04, 1.65271239e-05,
        2.07673818e-06],
       [3.25999461e-02, 3.27711239e-02, 1.64716002e-02, 5.51936342e-03,
        1.38708621e-03, 2.78873920e-04, 4.67230413e-05, 6.70976821e-06,
        8.43125032e-07],
       [9.92632251e-03, 9.97844424e-03, 5.01541982e-03, 1.68058503e-03,
        4.22352386e-04, 8.49140199e-05, 1.42266486e-05, 2.04305010e-06,
        2.56722234e-07],
       [2.41796421e-03, 2.43066060e-03, 1.22171183e-03, 4.09375622e-04,
        1.02881299e-04, 2.06843028e-05, 3.46548553e-06, 4.97668901e-07,
        6.2

# Calculating Probabilities of Dixon-Coles Model

In [12]:
ft1, ftx, ft2, ft_score = [], [], [], []
over_15, over_25, under_35, under_45, btts = [], [], [], [], []
ht1, htx, ht2, ht_score, ht_over05, ht_under15 = [], [], [], [], [], []
ho05, ao05, ho15, ao15, hu25, au25 = [], [], [], [], [], []

# Helper function to calculate total goals for each score
def total_goals(i, j):
    return i + j

for i in range(len(next_matches)):
    my_matrix = full_time_matrices[i]
    ht_matrix = half_time_matrices[i]

    ft1.append(round(np.sum(np.tril(my_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    ftx.append(round(np.sum(np.diag(my_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ft2.append(round(np.sum(np.triu(my_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)
    
    max_score = np.unravel_index(np.argmax(my_matrix), my_matrix.shape) # Find the index of the maximum score
    home_goals, away_goals = max_score
    ft_score.append(f"{home_goals}-{away_goals}") # Format the score as 'home-away'

    # Calculate the probabilities
    over_15.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 1.5]) * 100, 2))
    over_25.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 2.5]) * 100, 2))
    under_35.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 3.5]) * 100, 2))
    under_45.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 4.5]) * 100, 2))

    # Calculate BTTS (both teams to score and goals != 0)
    btts.append(round(np.sum([my_matrix[i, j] for i in range(1, my_matrix.shape[0]) for j in range(1, my_matrix.shape[1])]) * 100, 2)) 

    # Calculate statistics for Half Time
    ht1.append(round(np.sum(np.tril(ht_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    htx.append(round(np.sum(np.diag(ht_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ht2.append(round(np.sum(np.triu(ht_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)

    ht_max_score = np.unravel_index(np.argmax(ht_matrix), ht_matrix.shape) # Find the index of the maximum score
    ht_hogs, ht_awgs = ht_max_score
    ht_score.append(f"{ht_hogs}-{ht_awgs}") # Format the score as 'home-away'

    ht_over05.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) > 0.5]) * 100, 2))   
    ht_under15.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) < 1.5]) * 100, 2)) 

    ho05.append(round(np.sum(my_matrix[1:,:]) * 100, 2))
    ao05.append(round(np.sum(my_matrix[:,1:]) * 100, 2))
    ho15.append(round(np.sum(my_matrix[2:,:]) * 100, 2))
    ao15.append(round(np.sum(my_matrix[:,2:]) * 100, 2))
    hu25.append(round(np.sum(my_matrix[:3,:]) * 100, 2))
    au25.append(round(np.sum(my_matrix[:,:3]) * 100, 2))
    

# Combine lists into a DataFrame
final_results = pd.DataFrame({
    'League': next_matches['League'], 'Home': next_matches['Home'], 'Away': next_matches['Away'],
    'FT1': ft1, 'FTX': ftx, 'FT2': ft2, 'FTR': ft_score,
    'DC1X': [x + y for x, y in zip(ft1, ftx)], 'DC12': [x + y for x, y in zip(ft1, ft2)], 'DCX2': [x + y for x, y in zip(ftx, ft2)],
    '1.5O': over_15, '2.5O': over_25, '3.5U': under_35, '4.5U': under_45, 'BTTS': btts,
    'HT1': ht1, 'HTX': htx, 'HT2': ht2, 'HTR': ht_score,
    'HTDC1X': [x + y for x, y in zip(ht1, htx)], 'HTDC12': [x + y for x, y in zip(ht1, ht2)], 'HTDCX2': [x + y for x, y in zip(htx, ht2)],
    'HT0.5O': ht_over05, 'HT1.5U': ht_under15, 'H0.5O':ho05, 'A0.5O':ao05, 'H1.5O':ho15, 'A1.5O':ao15, 'H2.5U':hu25, 'A2.5U':au25
})

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str):
        return ''  # Return empty string for NaN values
    elif value > threshold:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = final_results.style.applymap(highlight_values)
styled_df.to_excel(given_date + ".xlsx", index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

,League,Home,Away,FT1,FTX,FT2,FTR,DC1X,DC12,DCX2,1.5O,2.5O,3.5U,4.5U,BTTS,HT1,HTX,HT2,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O,HT1.5U,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U
0,England2,Cardiff City,Preston,39.260000,32.120000,28.620000,1-1,71.380000,67.880000,60.740000,66.800000,38.350000,81.480000,92.500000,46.340000,30.170000,45.060000,24.730000,0-0,75.230000,54.900000,69.790000,64.720000,72.550000,70.420000,63.400000,34.380000,26.620000,87.560000,91.870000
1,England2,Hull City,Watford,29.260000,30.380000,40.350000,1-1,59.640000,69.610000,70.730000,71.850000,44.310000,76.840000,89.780000,51.110000,28.590000,49.740000,21.650000,0-0,78.330000,50.240000,71.390000,58.180000,78.690000,67.010000,73.820000,30.430000,38.740000,89.860000,84.770000
2,England2,Millwall,Sheffield Utd,20.120000,40.030000,39.850000,0-0,60.150000,59.970000,79.880000,43.100000,17.220000,94.360000,98.480000,26.050000,6.760000,55.460000,37.760000,0-0,62.220000,44.520000,93.220000,47.710000,86.330000,42.060000,58.430000,10.440000,21.940000,98.190000,94.070000
3,England2,QP Rangers,Oxford Utd,43.250000,30.120000,26.630000,1-1,73.370000,69.880000,56.750000,71.670000,44.110000,77.000000,89.880000,50.430000,35.880000,41.000000,23.020000,0-0,76.880000,58.900000,64.020000,70.080000,66.590000,75.200000,64.910000,40.620000,28.160000,83.490000,91.080000
4,England2,West Brom,Coventry City,55.500000,29.150000,15.350000,1-0,84.650000,70.850000,44.500000,65.260000,36.910000,82.530000,93.080000,39.560000,51.780000,34.300000,13.490000,1-0,86.080000,65.270000,47.790000,75.410000,59.150000,77.550000,49.120000,44.010000,14.740000,81.040000,96.870000
